<a href="https://colab.research.google.com/github/DeepSmeag/ML_Colab/blob/main/AND%2BXORperceptrons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/DeepSmeag/ML_Colab

fatal: destination path 'ML_Colab' already exists and is not an empty directory.


First we load the project so we get access to the files. Then we read the files of interest

In [3]:
def read_file(file_path):
  dataset=[]
  with open(file_path, 'r') as f:
    lines = f.readlines()
    for line in lines:
      line = line.strip()
      parts = line.split(',')
      x0,x1,y1,y2 = float(parts[0]), float(parts[1]), float(parts[2]), float(parts[3])
      dataset.append((x0,x1,y1,y2))
    return dataset

dataset = read_file('ML_Colab/datasetAND+XOR.csv')
"""it could also work with numpy and/or pandas to read the whole csv file at the
same time, probably more efficient too
"""


'it could also work with numpy and/or pandas to read the whole csv file at the\nsame time, probably more efficient too\n'

We now have our dataset ready for use, next let's go and make the vectors for use with our network

In [4]:
import numpy as np
import csv
import pandas

In [73]:
#first layer is input, meaning we take the first 2 columns of the dataset
num_trains = len(dataset)
X = np.array(dataset)[:,0:2]
#now we add the bias to every training instance
X = np.append(X, np.ones((num_trains,1)), axis=1)
# now we take the outputs
Y = np.array(dataset)[:,2:4]

In [146]:
#now we create Numpy arrays so that we have our weights
#the first layer
W1 = np.random.uniform(-1,1,(3,3))
#the second
W2 = np.random.uniform(-1,1,(4,2))
print("W1:\n",W1,"\n")
print("W2:\n",W2,"\n")

W1:
 [[ 0.74165745 -0.53617734  0.52749035]
 [ 0.02613715  0.8193988  -0.7573968 ]
 [-0.7655541   0.19871105 -0.75346756]] 

W2:
 [[-0.79246294 -0.46252752]
 [ 0.2940028  -0.82842218]
 [ 0.61026612  0.01378462]
 [ 0.18540812 -0.14086743]] 



In [160]:
#Now we're getting to the fun stuff
#First, defining the necessary functions

def sigmoid(x):
  return 1/(1 + np.exp(-x))

#Forward Propagation
def forward_prop():
  global W1, W2
  #calculating the steps
  z2 = np.matmul(X, W1)
  a2 = sigmoid(z2)
  a2 = np.insert(a2, 3, np.ones((num_trains,)), axis = 1)
  z3 = np.matmul(a2, W2)
  y_hat = sigmoid(z3)
  return z2, a2, z3, y_hat



In [169]:
#Ok, so that was forward propagation
#Now, the NN should also be able to learn, so we have to implement Backprop
#Since we're guessing at best at the moment as to the correct outputs
#We have to define several functions for Backprop to work
#These include the partial derivatives with which we'll update the weights matrices W1 and W2
#And also the backprop itself which will go over epochs or whatever we may call it

# matrices sizes
# X = num_trains x 3
# W1 = 3 x 3
# Z2 = num_trains x 3
# A2 = num_trains x 4 by adding a collumn of ones at the end
# W2 = 4 x 2
# Z3 = num_trains x 2
# A3 = Y_hat = num_trains x 2


def sigmoid_gradient(x):
  return np.multiply(sigmoid(x),(1-sigmoid(x)))
def derivative(z2, a2, z3, y_hat, training_point):
  dz3 = sigmoid_gradient(z3[training_point, :]) # 1 x 2
  dw2 = np.matmul(np.matrix(a2[training_point, :]).T, np.matrix(dz3)) # 4 x 2
  dz2 = np.matrix(sigmoid_gradient(z2[training_point, :])) # 1 x 3

  
  

  da2 = np.matmul(dz3, W2.transpose()) # 1 x 4
  da2 = np.delete(da2, 3) # drop last column because it's for bias, 1 x 3

  temp1 = np.multiply(da2, dz2) # 1 x 3
  
  dw1 = np.matmul(np.matrix(X[training_point, :]).T, temp1) # 3 x 3
  return dw1, dw2

def backprop(num_trains, learning_rate):
  global W1, W2
  z2, a2, z3, y_hat = forward_prop()
  for training_point in range(1, num_trains, 4):
    W1 = np.genfromtxt('ML_Colab/AND+XORmodelW1.csv', delimiter=',')

    W2 = np.genfromtxt('ML_Colab/AND+XORmodelW2.csv', delimiter=',')
    print("current_training_point: ", training_point+1, "\n")
    
    dw1, dw2 = derivative(z2, a2, z3, y_hat, training_point)
    print(W1)
    W1 = np.subtract(W1, learning_rate * dw1)
    print(W1)
    W2 = np.subtract(W2, learning_rate * dw2)
    error = y_hat[training_point, :] - Y[training_point, :]
    print("\n",error)
    error = np.power(error, 2)
    error = np.sum(error)
    print("error:", error)
    print(y_hat[training_point, :])
    with open("ML_Colab/AND+XORmodelW1.csv", 'w') as fw1:
      np.savetxt(fw1 , W1 , fmt='%s', delimiter=',')
    with open("ML_Colab/AND+XORmodelW2.csv", 'w') as fw2:
      np.savetxt(fw2 , W2 , fmt='%s', delimiter=',')

backprop(num_trains, 0.001)

current_training_point:  2 

[[ 5.29681321  1.06846011 -1.45816033]
 [ 4.25036942  2.17193181 -2.29595142]
 [32.18151198 24.39967004 -5.73868591]]
[[ 5.29681321  1.06846011 -1.45816033]
 [ 4.25036942  2.17193181 -2.29595142]
 [32.18151198 24.39967004 -5.73868591]]

 [[ 4.83229199e-78 -1.00000000e+00]]
error: 1.0
[[4.83229199e-78 3.70938125e-67]]
current_training_point:  6 

[[ 5.29681321  1.06846011 -1.45816033]
 [ 4.25036942  2.17193181 -2.29595142]
 [32.18151198 24.39967004 -5.73868591]]
[[ 5.29681321  1.06846011 -1.45816033]
 [ 4.25036942  2.17193181 -2.29595142]
 [32.18151198 24.39967004 -5.73868591]]

 [[ 4.83229199e-78 -1.00000000e+00]]
error: 1.0
[[4.83229199e-78 3.70938125e-67]]
current_training_point:  10 

[[ 5.29681321  1.06846011 -1.45816033]
 [ 4.25036942  2.17193181 -2.29595142]
 [32.18151198 24.39967004 -5.73868591]]
[[ 5.29681321  1.06846011 -1.45816033]
 [ 4.25036942  2.17193181 -2.29595142]
 [32.18151198 24.39967004 -5.73868591]]

 [[ 4.83229199e-78 -1.00000000e+00]]


In [166]:
new_w1 = np.genfromtxt('ML_Colab/AND+XORmodelW1.csv', delimiter=',')

new_w2 = np.genfromtxt('ML_Colab/AND+XORmodelW2.csv', delimiter=',')
def test(W1, W2):
  
  #calculating the steps
  z2 = np.matmul(X, W1)
  a2 = sigmoid(z2)
  a2 = np.insert(a2, 3, np.ones((num_trains,)), axis = 1)
  z3 = np.matmul(a2, W2)
  y_hat = sigmoid(z3)
  return z2, a2, z3, y_hat
Null, Null, Null, y_hat = test(new_w1, new_w2)
y_hat = np.rint(y_hat)
print(y_hat)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [79]:
Y

array([[0., 0.],
       [0., 1.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [1., 0.]])